In [1]:
import pandas as pd
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np

In [2]:
games_df = pd.read_csv('games_df.zip',compression = 'zip')
games_df['Increment']=games_df['Increment'].astype('object')

In [3]:
games_df=pd.get_dummies(games_df,columns=['Result','Time_Control','Increment','Termination'])

In [4]:
features= games_df.drop('Average_Elo',axis=1).describe().columns.to_list()[2:]

In [5]:
Y = games_df['Average_Elo']
X = games_df[features]

In [6]:
X_train,X_test,Y_train,Y_test = tts(X,Y,test_size=0.33, random_state=73)

In [29]:
#Modelo Linear
linear = LassoCV()
linear.fit(X_train,Y_train)
print(linear.score(X_train,Y_train), linear.alpha_)

0.21874669315378736 2.3416611423034177


In [31]:
coef=linear.coef_
coef_dic = {param:coefi for param,coefi in zip(features,coef)}
coef_dic

{'N_Moves': 11.740795641303935,
 'Open_N_Moves': 31.772157847168724,
 'White_Open_Time': -4.194668643067498,
 'Black_Open_Time': -8.03661058935981,
 'White_last_10_moves': 1.6337831560880944,
 'Black_last_10_moves': 2.814138874138578,
 'White_Time_Scramble': -14.578180187613471,
 'Black_Time_Scramble': -7.34220696506846,
 'White_Eval': 0.9359929673553932,
 'Black_Eval': 0.9577314764856011,
 'White_Open_Eval': 19.879918745233482,
 'Black_Open_Eval': 28.46745217376436,
 'Result_0-1': -0.0,
 'Result_1-0': 0.0,
 'Time_Control_Blitz': -53.59245127800955,
 'Time_Control_Bullet': 81.95300960873355,
 'Time_Control_Classical': 0.0,
 'Time_Control_Rapid': -29.278415166725143,
 'Increment_0.0': 47.17065778526478,
 'Increment_1.0': -0.0,
 'Increment_2.0': -0.0,
 'Increment_3.0': -26.63916822035841,
 'Increment_4.0': 0.0,
 'Increment_5.0': 0.0,
 'Increment_6.0': 0.0,
 'Increment_7.0': 0.0,
 'Increment_8.0': 0.0,
 'Increment_9.0': 0.0,
 'Increment_10.0': 0.0,
 'Increment_11.0': -0.0,
 'Increment_12.

In [28]:
#Modelo de floresta básico
rf=RandomForestRegressor()
rf.fit(X_train,Y_train)

RandomForestRegressor(n_estimators=1000)

In [29]:
rf.score(X_test,Y_test)

0.3160142476974076

In [27]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [30]:
#Criando faixa para os hiperparametros
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
# Número de features consideradas para cada split
max_features = ['sqrt','auto']
#Nível da árvpre
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
#Número de amostras para split
min_samples_split = [2, 5, 10]
#Número de amostras para cada nodo
min_samples_leaf = [1, 2, 4]
bootstrap = [True]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


In [31]:

rf = RandomForestRegressor()
#Procura aleatórioa, cv de 5 folds seria melhor, mas aumenta o tempo de execução, por isso 3
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 80, cv = 3, random_state=73, n_jobs = -1)# Fit the random search model
rf_random.fit(X_train, Y_train)

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=80,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['sqrt', 'auto'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500, 600, 700, 800,
                                                         900, 1000]},
                   random_state=73)

In [32]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 50,
 'bootstrap': True}

In [33]:
rf_random.best_score_

0.3168387724613463

Houve uma mehora baixa com realção ao modelo de floresta sem nenum ajuste. Mas assim como o básico tem um desembenho bem melhor que o linear, Porém ainda e um desempeho baixo, explicando apenas 31% da variância

In [47]:
#Realizando grid search a aprtir dos resultados da random search
param_grid = {
    'bootstrap': [True],
    'max_depth': [40, 50, 60, 70],
    'max_features': ['auto'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [500, 600,700]
}# Create a based model
rf = RandomForestRegressor()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1)
grid_search.fit(X_train,Y_train)

KeyboardInterrupt: 

In [37]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 12,
 'n_estimators': 600}

In [38]:
grid_search.best_score_

0.31702976883446055

Houve uma melhora mínima, mas tanto o número de splits como nível aumentaram até chegar no máximo, irei realizar uma nova bsca para checar se ainda não há mais epaço para aumento desses parâmetros

In [40]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [70,80,90, 100],
    'max_features': ['auto'],
    'min_samples_leaf': [ 4],
    'min_samples_split': [ 12,14,16],
    'n_estimators': [600]
}# Create a based model
rf = RandomForestRegressor()# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1)
grid_search.fit(X_train,Y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [70, 80, 90, 100],
                         'max_features': ['auto'], 'min_samples_leaf': [4],
                         'min_samples_split': [12, 14, 16],
                         'n_estimators': [600]})

In [41]:
grid_search.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 12,
 'n_estimators': 600}

In [42]:
grid_search.best_score_

0.31664211314874796

In [45]:
a={'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 12,
 'n_estimators': 600}
b={'bootstrap': True,
 'max_depth': 80,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 12,
 'n_estimators': 600}

rf1 = RandomForestRegressor()
rf2 = RandomForestRegressor()
rf1.set_params(**a)
rf2.set_params(**b)


RandomForestRegressor(max_depth=80, min_samples_leaf=4, min_samples_split=12,
                      n_estimators=600)

In [46]:
rf1.fit(X_train,Y_train)
rf2.fit(X_train,Y_train)
print(rf1.score(X_test,Y_test),rf2.score(X_test,Y_test))


0.3180800081927251 0.3179440749299579


A diferença entre os resultados do teste entre o modelo do 1º gridsearch e do 2º é mínimo.

Tantos o modelo de regressão linear por Lasso como o modelo de random forest possuem desempenho muito baixo. Também gostaria de utilizar um modelo MARS mas estou tendo problemas na instalação do módulo py-earth (assim que resolver realizo um teste com esses modelo), também penso em realizar um modelo de gradient boosting. Porém uma outra bia para melhorar os resultados seria realizar um novo processo de feature enginering para tentar extrair informações mais relevantes ao problema.